## Overview
This notebook tests importing and running the scripted version of the continuity checker.  The use case in mind is that one would initialize the station in the primary measurement notebook and then call this script (i.e. eliminate the need to re-initialize the instruments in separate notebooks since the MDAC takes so long to connect to.

## Imports

In [3]:
import qcodes as qc
import numpy as np
from matplotlib import pyplot as plt
import logging
import matplotlib.pyplot as plt
import time

from qcodes.instrument_drivers.tektronix.Keithley_2600_channels import Keithley_2600
from StationQ.MDAC.Driver.MDAC import MDAC

from v0_utils.qcodes_tools import \
instrument_factory

from Continuity_checker import continuity_checker
from wiring_maps import channel_mapper

67


In [2]:
station = qc.Station()

mdac1 = instrument_factory(
    MDAC, "mdac1", address='ASRL7::INSTR')
station.add_component(mdac1)

mdac2 = instrument_factory(
    MDAC, "mdac2", address='ASRL4::INSTR')
station.add_component(mdac2)

keithley = instrument_factory(Keithley_2600, "keithley", address='TCPIP::169.254.0.1')
station.add_component(keithley)

C:\ProgramData\Anaconda3\envs\qcodes4\lib\site-packages\pyvisa\ctwrapper\functions.py:1649: VisaIOWarning: VI_WARN_NSUP_ATTR_STATE (1073676420): Although the specified state of the attribute is valid, it is not supported by this implementation.
  return library.viSetAttribute(session, attribute, attribute_state)


Connected to: QNL Sydney MDAC (serial:0x303035313535510D0032001D, firmware:1.4) in 4.78s
Connected to: QNL Sydney MDAC (serial:0x303035313535510D0031001D, firmware:1.4) in 4.87s
Connected to: Keithley Instruments Inc. 2614B (serial:4349231, firmware:3.2.2) in 0.18s


'keithley'

In [3]:
from Continuity_checker import continuity_checker

In [5]:
R_dict = continuity_checker(station.mdac1, station.mdac2, station.keithley, probe_connection=False)

Total continuity check time = 74.3768699169159s


In [7]:
for key in R_dict:
    if R_dict[key] < 1e6:
        print(key,': ',R_dict[key])

2-14 :  103253.0
3-2 :  13564.560000000001
3-3 :  13573.939999999999
3-7 :  17633.31
3-8 :  13666.470000000001
3-9 :  13541.009999999998
3-10 :  13644.689999999999
3-11 :  13602.159999999998
3-20 :  13637.059999999998
3-21 :  13687.9
3-22 :  13696.340000000002
3-23 :  13654.219999999998
3-24 :  16942.15
4-1 :  13568.98
4-7 :  18380.420000000002
4-9 :  13597.060000000001
4-11 :  18052.06
4-12 :  13620.569999999998
4-16 :  13698.320000000002
4-24 :  13722.65
4-25 :  13673.170000000002


Seems to work as expected

In [15]:
R_dict['4-20']

14944713300.0

In [9]:
BG1 = channel_mapper('4-16', station.mdac1.channels, station.mdac2.channels, probe_connection=False)

In [11]:
BG2 = channel_mapper('4-1', station.mdac1.channels, station.mdac2.channels, probe_connection=False)

In [19]:
BG1.voltage(0)
BG2.voltage(0)

BG1.dac_output('open')
BG2.dac_output('open')

BG1.bus('close')

BG1.microd('close')
BG2.microd('close')

In [20]:
station.keithley.smub.res()

1402580000.0

In [21]:
BG2.gnd('close')

In [22]:
station.keithley.smub.res()

13714.2

In [23]:
BG1.microd('open')
BG2.microd('open')
BG2.gnd('open')
BG1.bus('open')

In [71]:
S1 = channel_mapper('4-11', station.mdac1.channels, station.mdac2.channels, probe_connection=False)
S2 = channel_mapper('3-7', station.mdac1.channels, station.mdac2.channels, probe_connection=False)
D1 = channel_mapper('4-11', station.mdac1.channels, station.mdac2.channels, probe_connection=False)
D2 = channel_mapper('3-24', station.mdac1.channels, station.mdac2.channels, probe_connection=False)

In [75]:
#Float unused pins
for ch in [S1, S2, D1, D2]:
    ch.voltage(0)
    ch.dac_output('open')
    ch.smc('open')
    ch.gnd('open')
    ch.bus('open')
    ch.microd('open')
S1.bus('close')
D2.gnd('close')
S1.microd('close')
D1.microd('close')
S2.microd('close')
D2.microd('close')
print(station.keithley.smub.res()/1e3)

266.926


In [41]:
station.keithley.smua.res()

36433300000.0

In [77]:
for ch in station.mdac1.channels:
    ch.microd('open')
    ch.bus('open')

for ch in station.mdac2.channels:
    ch.microd('open')
    ch.bus('open')
    